In [66]:
# TODO: Seperate flex back out and treat as its own feature by points total. Looking at contributions by RB makes less sense if there were weeks in which people didn't field an RB in their flex

In [39]:
%run ../utils.ipynb

In [54]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

from warnings import simplefilter

In [41]:
simplefilter(action='ignore', category=FutureWarning)

In [42]:
data_dir = f"{project_dir}data\\matchup_summary\\"

In [43]:
df = pd.read_csv(data_dir + 'weekly_position_scoring_with_matchup_result.csv')

In [44]:
num_cols = list(df.select_dtypes(include=np.number).columns)
num_cols.remove('points_for')

In [45]:
for i in num_cols:
    df[f"{i}_percentage"] = round((df[i] / df['points_for']) * 100, 2)

In [46]:
df = df[df['season_week'].str.contains('2023')]

In [49]:
X = df[['QB_points', 'DEF_points', 'WR_total_points', 'RB_total_points','TE_total_points']]
y = df['win']

In [50]:
lr = LogisticRegressionCV(cv=5, random_state=2024)

In [51]:
lr.fit(X, y)

LogisticRegressionCV(cv=5, random_state=2024)

In [52]:
np.mean(cross_val_score(lr, X, y))

0.767979797979798

In [53]:
np.exp(lr.coef_)

array([[1.07180236, 1.07300013, 1.08880538, 1.09308758, 1.06913095]])

In [55]:
sc = StandardScaler()

In [56]:
X_sc = sc.fit_transform(X)

In [64]:
np.mean(df['WR_total_points'])

30.11794642857143

In [63]:
np.std(df['WR_total_points'])

13.437083856572402

In [60]:
lr.fit(X_sc,y)

LogisticRegressionCV(cv=5, random_state=2024)

In [61]:
np.mean(cross_val_score(lr, X_sc, y))

0.7768686868686869

In [62]:
np.exp(lr.coef_)

array([[1.69806365, 1.68320693, 2.81815107, 2.95136091, 1.55061525]])